# Installing Magenta
Taken from their [Getting Started](https://colab.research.google.com/notebooks/magenta/hello_magenta/hello_magenta.ipynb#scrollTo=PfRDVhNs3UFx) notebook.

This will take a few minutes to run.

In [ ]:
#@test {"output": "ignore"}
print('Installing dependencies...')
!apt-get update -qq && apt-get install -qq libfluidsynth1 fluid-soundfont-gm build-essential libasound2-dev libjack-dev
!pip install -qU pyfluidsynth pretty_midi

!pip install -qU magenta

# Hack to allow python to pick up the newly-installed fluidsynth lib. 
# This is only needed for the hosted Colab environment.
import ctypes.util
orig_ctypes_util_find_library = ctypes.util.find_library
def proxy_find_library(lib):
  if lib == 'fluidsynth':
    return 'libfluidsynth.so.1'
  else:
    return orig_ctypes_util_find_library(lib)
ctypes.util.find_library = proxy_find_library

print('Importing libraries and defining some helper functions...')
from google.colab import files

import magenta
import note_seq
import tensorflow

print('🎉 Done!')
print(magenta.__version__)
print(tensorflow.__version__)

Installing dependencies...
Selecting previously unselected package fluid-soundfont-gm.
(Reading database ... 160772 files and directories currently installed.)
Preparing to unpack .../fluid-soundfont-gm_3.1-5.1_all.deb ...
Unpacking fluid-soundfont-gm (3.1-5.1) ...
Selecting previously unselected package libfluidsynth1:amd64.
Preparing to unpack .../libfluidsynth1_1.1.9-1_amd64.deb ...
Unpacking libfluidsynth1:amd64 (1.1.9-1) ...
Setting up fluid-soundfont-gm (3.1-5.1) ...
Setting up libfluidsynth1:amd64 (1.1.9-1) ...
Processing triggers for libc-bin (2.27-3ubuntu1.2) ...
/sbin/ldconfig.real: /usr/local/lib/python3.7/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link

     |████████████████████████████████| 5.6MB 23.5MB/s 
     |████████████████████████████████| 51kB 6.1MB/s 
     |████████████████████████████████| 1.4MB 30.6MB/s 
     |████████████████████████████████| 1.6MB 39.7MB/s 
     |████████████████████████████████| 215kB 42.5MB/s 
     |█████████████████████████

/usr/local/lib/python3.7/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/usr/local/lib/python3.7/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit


🎉 Done!
2.1.3
2.5.0


# MusicVAE'
This model will serve as the generator for new MIDI as well as the soundtrack variations.

Begin with downloading and extracting the pre-trained model that produces Drums, Melody, and Bass at once.

This will also take a few minutes to run.

In [ ]:
# See the list of checkpoints: https://github.com/magenta/magenta/tree/master/magenta/models/music_vae#pre-trained-checkpoints
import os

if not os.path.isfile("/content/hierdec-trio_16bar.tar"):
    os.system("curl -o hierdec-trio_16bar.tar https://storage.googleapis.com/magentadata/models/music_vae/checkpoints/hierdec-trio_16bar.tar")

if not os.path.isfile("/content/hierdec-trio_16bar.ckpt.index"):
    os.system("tar -xvf /content/hierdec-trio_16bar.tar")

In [ ]:
# Initialize the model.
from magenta.models.music_vae import configs
from magenta.models.music_vae.trained_model import TrainedModel

music_vae = TrainedModel(
      configs.CONFIG_MAP['hierdec-trio_16bar'], 
      batch_size=4, 
      checkpoint_dir_or_path='/content/hierdec-trio_16bar.ckpt')

INFO:tensorflow:Building MusicVAE model with BidirectionalLstmEncoder, HierarchicalLstmDecoder, and hparams:
{'max_seq_len': 256, 'z_size': 512, 'free_bits': 256, 'max_beta': 0.2, 'beta_rate': 0.0, 'batch_size': 4, 'grad_clip': 1.0, 'clip_mode': 'global_norm', 'grad_norm_clip_to_zero': 10000, 'learning_rate': 0.001, 'decay_rate': 0.9999, 'min_learning_rate': 1e-05, 'conditional': True, 'dec_rnn_size': [1024, 1024], 'enc_rnn_size': [2048, 2048], 'dropout_keep_prob': 1.0, 'sampling_schedule': 'constant', 'sampling_rate': 0.0, 'use_cudnn': False, 'residual_encoder': False, 'residual_decoder': False, 'control_preprocessing_rnn_size': [256]}
INFO:tensorflow:
Encoder Cells (bidirectional):
  units: [2048, 2048]

INFO:tensorflow:
Hierarchical Decoder:
  input length: 256
  level output lengths: [16, 16]

INFO:tensorflow:
Decoder Cells:
  units: [1024, 1024]

INFO:tensorflow:
Decoder Cells:
  units: [1024, 1024]

INFO:tensorflow:
Decoder Cells:
  units: [1024, 1024]



/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/legacy_tf_layers/core.py:171: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  warnings.warn('`tf.layers.dense` is deprecated and '
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1692: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1700: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
INFO:tensorflow:Restoring parameters from /content/hierdec-trio_16bar.ckpt


# MIDI Generation
Once the models have been loaded, it's a matter of calling the `sample()` function to generate new note sequences.

In [ ]:
from google.colab import files

def download_midi(note_sequence, filename):
    note_seq.sequence_proto_to_midi_file(note_sequence, filename)
    files.download(filename)

generated = music_vae.sample(n=1, length=80, temperature=0.4)

note_seq.plot_sequence(generated[0])
note_seq.play_sequence(generated[0], synth=note_seq.fluidsynth)

download_midi(generated[0], "trio_sample.mid")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# upload 2 midi files and interpolate them
upload = False

if upload:
    uploaded_files = files.upload() # this puts it in file storage too


Saving trio_sample(7).mid to trio_sample(7) (1).mid


In [ ]:
import random

for i in range(50):
    t = round(random.random(), 1)
    print(t)
    #new_midi = music_vae.sample(n=1, length=80, temperature=t)
    break

